# Python basics

* Type system
* Magic Methods
* Protocols (iterators, generators)
* GIL
* Sync vs Async
* Multithreading/ MUltiprocessing

## Warming activities

1. Python has some predefined values, classes and function, those are in the `__builtins__` package and automatically available for the user. Lets explore them in a Python REPL:
   1. dir() is quite usefull it shows the attributes of the object sent as an argument, if no arguments are provided it shows the defined names in the current scope. Run `dir()` and `dir(__builtins__)`.
   2. help() renders the documentation for any object (it does that by rendering the class docstring). Try `help(dir)`
   3. type() receives an instance and returns the class that instance belongs to. Try `type(1)`, `type(list)`, `type(type(1))`.
2. Some of the builtins are Python types, like int, float, str, bool, list, set, dict, tuple.
   1. Not an exercise: Simple types like int an float are quite intuitive, just let's say there is a regular division `/` that returns float even if both operators are int. There is also an integer division `//`. So `type(42 / 2) is float` but `type(42 // 2) is int` -`is` operator is for identity check, not comparision. Only use it for types and for None-
   2. Collections and strings are more interesting, there are mutable collections -list, dict, set- and inmutable collections -tuple, str, bytes-.
      1. Create a list with numbers, and a tuple with the same numbers.
      2. Try to append a new number to each type.
      3. Create a `set()` and try to add inmutable instances first and then mutable instances. Investigate why it doesn't work with inmutable collections, compare the case with `dict()` keys.
      5. As in some other languages, collections can be created by extension and some also by comprehension.
         1. Create a list of the squares of the first 20 integer numbers by extension by iterating in a range() object and appending data.
         2. Create the same list by extension by passing the range() object as argument to list().
         3. Create yet again the same list by comprehension.
         4. Explore how to create sets and dicts by comprehension.

---

**Fun fact** int instances in Python never overflows. An `int` instance starts its life as 64bits int but if the number you want to store doesn't fit it grows enabling to have ridiculously big integers -as far as system heap memory allows it-

---

## Protocols

### Special methods (dunder methods)

See: https://docs.python.org/3/reference/datamodel.html#special-method-names

This methods allows to better integrate our custom classes instances with the languague and its operators, some have a default behaviour if we do not override them and others are only defined when needed.

Normally dunder methods are not called directly, instead those are called when a given operation is done on the instance -eg. accesing by subscripting, deleting, calling as a function, comparing, etc-

When we use dunder methods there is no mandatory to inherit of any specific class, but it can be done using a Protocol subclass for a more strict type checking if wanted. See https://typing.python.org/en/latest/spec/protocol.html.

For example lets take the `repr()` builtin and the `>` operator, those use the `__repr__()` and `__gt__()` dunder methods. All objects have a default definition for `__repr__()`, this implementation retuns a basic description and the `id()` of the instance in hexadecimal. But not all have a `__gt__()`.
 

In [32]:
class A:
    pass

a = A()
b = A()
print(f"{a.__repr__() = }")
print(f"{a.__gt__(b) = }")
try:
    print(a > b)
except Exception as e:
    print(f"a > b raises: {type(e)}: {e}")

a.__repr__() = '<__main__.A object at 0x7fd2e4198c50>'
a.__gt__(b) = NotImplemented
a > b raises: <class 'TypeError'>: '>' not supported between instances of 'A' and 'A'


### Activities

1. Create a custom class that behave similarly to a dict but also allows to access the items as attributes -like a mix between dict and namedtuple-. Implement only setting and getting, do not bother with other methods.   
```python
class AttrDict:
    def __init__(self):
        super().__setattr__("dict", {})  # This is a help and a clue
    # implement the rest of the class ...
    
# Usage
d = AttrDict()
d["name"] = "Eric"
d["last_name"] = "Idle"

print(d.name)
print(d["name"])
d.name = "Terry"
d.last_name = "Jones"
print(d.name)
print(d.last_name)
```
2. If you create two identical instances of AttrDict() with the same values, the `==` comparision returns `False`, if you put them in a set, then the set has two elements, but by definition it only should have one. Try to fix it to work as intended by overriding `__hash__` and `__eq__`.

There is a useful example in `__hash__` documentation: https://docs.python.org/3/reference/datamodel.html#object.__hash__

Example code:
```python
a = AttrDict()
b = AttrDict()
a["name"] = "Eric"
a["last_name"] = "Idle"
a["age"] = 82
b["name"] = "Eric"
b["last_name"] = "Idle"
b["age"] = 82

a == b  # This is False but it should be True
a == a  # This returns True correctly
{a, b}  # This returns a set with two equal objects but set items should be unique instead
```

3. There are two main ways of creating a generator, by using `yield` statement in a function and by creating a generator by comprehension. Create a generator object that generates the square root of each integer number starting from 0 and without any superior limit. Use both ways to define the generator.

Example code:
```python
from math import sqrt
## Implement generator here
## ...
square_root_gen = # ...

for sqr in square_root_gen:
    print(srq)
    if sqr > 5:
        break
```
**Note**: Consider to use `itertools.count` at least for the generator by comprehension version.

## Sync / Async

Since version 3.5 Python has specific syntax for coroutines with `async *` and `await` statements.

There are certain restrinctions and considerations when running async Python code:

* `await` can only be used inside async functions.
* Some scheduler is needed to run the code. Tipically asyncio provided with Python std lib is used, altough third party options exist like curio and trio.
* Mixing blocking sync operations with async should be avoided if posible to avoid blocking the complete application. For that different libraries provide async abstractions for networking, DB operations and I/O in general.
* REPL and debugger do not have a nice way to run async functions. Instead jupyter is a good alternative to easily run async functions for small experiments

Many traditional Python frameworks have some degree of async support:

* FastAPI has a mature support for async (https://fastapi.tiangolo.com/async/)
* Flask traditionally was only sync altough there is optional async support installing `flask[async]` (https://flask.palletsprojects.com/en/stable/async-await/)
* Django has partial async support (https://docs.djangoproject.com/en/5.2/topics/async/)

In the ORM field SQLAlchemy supports async but special care should be taken to avoid implicit operations like lazy loading relationships (https://docs.sqlalchemy.org/en/20/orm/extensions/asyncio.html#preventing-implicit-io-when-using-asyncsession) it is quirky but usable.
There are ORMs designed for async like TortoiseORM but I have no experience to give an opinion here. If in doubt SQLAlchemy is a well stablished stable and tested framework.

### Activities

1. Run the following code and analyze the execution time. Fix the bug in the code to minimize the execution time by allowing both tasks to execute.

In [ ]:
import asyncio
import time

async def task_one():
    """An asynchronous task that prints a message and sleeps."""
    print("Task One: Starting...")
    time.sleep(2 # Simulate some I/O-bound operation
    print("Task One: Finished!")

async def task_two():
    """Another asynchronous task that prints a message and sleeps."""
    print("Task Two: Starting...")
    time.sleep(1) # Simulate another I/O-bound operation
    print("Task Two: Finished!")

async def main():
    """The main coroutine to run the asynchronous tasks concurrently."""
    print("Main: Creating tasks...")
    # Create tasks to run concurrently
    start_time = time.time()
    await asyncio.gather(task_one(), task_two())
    print(f"Main: All tasks completed in {time.time() - start_time}.")

if __name__ == "__main__":
    asyncio.run(main())